MULTITHREAD

In [6]:
import os
import numpy as np
import cv2
import concurrent.futures

def txt_para_jpg(txt_path):
    """Converte um arquivo .txt contendo pixels em uma imagem JPG."""
    data = np.loadtxt(txt_path)
    
    # Normaliza manualmente para 0-255
    min_val, max_val = data.min(), data.max()
    img = ((data - min_val) / (max_val - min_val) * 255).astype(np.uint8)

    return img

def salvar_como_mapa_calor(img, output_path, colormap=cv2.COLORMAP_JET):
    """Aplica mapa de calor e salva a imagem em JPG."""
    heatmap = cv2.applyColorMap(img, colormap)
    output_path = output_path.replace(".png", ".jpg")  # Garantir extensão JPG
    cv2.imwrite(output_path, heatmap, [cv2.IMWRITE_JPEG_QUALITY, 95])

def processar_imagem(arquivo_path, output_dir, categoria, paciente, flip=False):
    """Processa uma única imagem, aplicando mapa de calor e realizando flip horizontal (se necessário)."""
    classe_dir = os.path.join(output_dir, categoria, paciente)
    os.makedirs(classe_dir, exist_ok=True)
    
    nome_arquivo = os.path.basename(arquivo_path).replace(".png", ".jpg").replace(".txt", ".jpg")
    nome_saida = os.path.join(classe_dir, nome_arquivo)
    
    # Carregar imagem
    if arquivo_path.endswith('.txt'):
        img = txt_para_jpg(arquivo_path)
    else:
        img = cv2.imread(arquivo_path, cv2.IMREAD_GRAYSCALE)
    
    # Aplicar e salvar mapa de calor
    salvar_como_mapa_calor(img, nome_saida)
    
    # Se for da classe Sick, salvar a versão flipada
    if flip:
        img_flip = cv2.flip(img, 1)  # Flip horizontal
        nome_base, ext = os.path.splitext(nome_saida)
        nome_saida_flip = f"{nome_base}F.jpg"  # Adiciona 'F' antes da extensão
        salvar_como_mapa_calor(img_flip, nome_saida_flip)

def processar_paciente(paciente_path, output_dir, categoria, paciente):
    """Processa todas as imagens de um paciente."""
    imagens = []
    for arquivo in os.listdir(paciente_path):
        if arquivo.lower().endswith(('.png', '.jpg', '.jpeg', '.txt')):
            imagens.append(os.path.join(paciente_path, arquivo))
    
    # Processar imagens em paralelo
    with concurrent.futures.ThreadPoolExecutor() as executor:
        for img_path in imagens:
            flip = (categoria == "Sick")  # Só flipar imagens de doentes
            executor.submit(processar_imagem, img_path, output_dir, categoria, paciente, flip)

input_dirs = [
    "/mnt/d/Documentos Gustavo/TCC/DMR - Database For Mastology Research - Visual Lab, UFF, Niterói, Brazil",
    "/mnt/d/Documentos Gustavo/TCC/DMR - Database For Mastology Research - Visual Lab, UFF, Niterói, Brazil-002",
    "/mnt/d/Documentos Gustavo/TCC/DMR - Database For Mastology Research - Visual Lab, UFF, Niterói, Brazil-003"
]

output_dir = "/mnt/d/Documentos Gustavo/TCC/processed_heatmaps/"
os.makedirs(output_dir, exist_ok=True)

# Contadores de imagens
countHealthy = 0
countSick = 0

for input_dir in input_dirs:
    for categoria in ["Healthy", "Sick"]:
        categoria_path = os.path.join(input_dir, categoria)
        if not os.path.exists(categoria_path):
            continue
        
        for paciente in os.listdir(categoria_path):
            for tipo_imagem in ["static", "dynamic"]:
                paciente_path = os.path.join(categoria_path, paciente, "visit_01", "images", "thermography", tipo_imagem)
                if os.path.exists(paciente_path):
                    processar_paciente(paciente_path, output_dir, categoria, paciente)
                    if categoria == "Healthy":
                        countHealthy += 1
                    elif categoria == "Sick":
                        countSick += 1

print("Healthy:", countHealthy)
print("Sick:", countSick)


Healthy: 766
Sick: 131


In [8]:
import os
import cv2
import numpy as np
from concurrent.futures import ThreadPoolExecutor

def aplicar_zoom(img, zoom_factor=1.1):
    """Aplica um leve zoom na imagem sem alterar a resolução."""
    h, w = img.shape[:2]  # Obtém altura e largura

    # Define novos limites para recorte central
    new_h, new_w = int(h / zoom_factor), int(w / zoom_factor)
    y1, x1 = (h - new_h) // 2, (w - new_w) // 2
    y2, x2 = y1 + new_h, x1 + new_w

    # Recorta e redimensiona para o tamanho original
    cropped = img[y1:y2, x1:x2]
    zoomed = cv2.resize(cropped, (w, h), interpolation=cv2.INTER_LINEAR)

    return zoomed

def processar_imagem_zoom(image_path):
    """Aplica zoom em uma imagem e salva com a letra 'Z' no nome."""
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    
    if img is None:
        print(f"Erro ao carregar {image_path}")
        return
    
    img_zoom = aplicar_zoom(img, zoom_factor=1.1)
    
    # Define o nome do arquivo de saída com "Z" antes da extensão
    pasta, nome_arquivo = os.path.split(image_path)
    base, ext = os.path.splitext(nome_arquivo)
    novo_nome = f"{base}Z{ext}"
    output_path = os.path.join(pasta, novo_nome)

    cv2.imwrite(output_path, img_zoom, [cv2.IMWRITE_JPEG_QUALITY, 95])
    print(f"Salvo: {output_path}")

def processar_pasta_sick(base_dir="/mnt/d/Documentos Gustavo/TCC/processed_heatmaps/Sick"):
    """Aplica zoom a todas as imagens dentro das subpastas de Sick."""
    if not os.path.exists(base_dir):
        print("Pasta não encontrada:", base_dir)
        return
    
    # Percorre todas as subpastas (p01, p02, ...)
    pacientes = [os.path.join(base_dir, p) for p in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, p))]
    
    if not pacientes:
        print("Nenhuma subpasta encontrada na pasta Sick.")
        return

    imagens = []
    for paciente in pacientes:
        for img in os.listdir(paciente):
            if img.lower().endswith(('.png', '.jpg', '.jpeg')):
                imagens.append(os.path.join(paciente, img))

    if not imagens:
        print("Nenhuma imagem encontrada para processar.")
        return

    # Processamento paralelo
    with ThreadPoolExecutor() as executor:
        for img_path in imagens:
            executor.submit(processar_imagem_zoom, img_path)

# Executa o script
processar_pasta_sick()


Salvo: /mnt/d/Documentos Gustavo/TCC/processed_heatmaps/Sick/p138/T0138.2.1.D.2013-09-06.00Z.jpgSalvo: /mnt/d/Documentos Gustavo/TCC/processed_heatmaps/Sick/p138/T0138.2.1.D.2013-09-06.03FZ.jpg

Salvo: /mnt/d/Documentos Gustavo/TCC/processed_heatmaps/Sick/p138/T0138.2.1.D.2013-09-06.05FZ.jpg
Salvo: /mnt/d/Documentos Gustavo/TCC/processed_heatmaps/Sick/p138/T0138.2.1.D.2013-09-06.07FZ.jpg
Salvo: /mnt/d/Documentos Gustavo/TCC/processed_heatmaps/Sick/p138/T0138.2.1.D.2013-09-06.05Z.jpg
Salvo: /mnt/d/Documentos Gustavo/TCC/processed_heatmaps/Sick/p138/T0138.2.1.D.2013-09-06.02FZ.jpg
Salvo: /mnt/d/Documentos Gustavo/TCC/processed_heatmaps/Sick/p138/T0138.2.1.D.2013-09-06.06Z.jpg
Salvo: /mnt/d/Documentos Gustavo/TCC/processed_heatmaps/Sick/p138/T0138.2.1.D.2013-09-06.01FZ.jpg
Salvo: /mnt/d/Documentos Gustavo/TCC/processed_heatmaps/Sick/p138/T0138.2.1.D.2013-09-06.03Z.jpg
Salvo: /mnt/d/Documentos Gustavo/TCC/processed_heatmaps/Sick/p138/T0138.2.1.D.2013-09-06.06FZ.jpg
Salvo: /mnt/d/Documentos